# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1.
### Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
URL = 'https://habr.com/ru/search/'
params = {
    'q': ['python', 'парсинг'],
}

In [3]:
res = requests.get(URL, params)
res.text

'<!DOCTYPE html>\n<html lang="ru" class="no-js">\n  <head>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8" />\n<meta content=\'width=1024\' name=\'viewport\'>\n<title>Результаты поиска по запросу «парсинг» / Хабр</title>\n\n\n\n\n\n<meta name=\'yandex-verification\' content=\'71593b225aeafc4e\' />\n<meta name=\'referrer\' content=\'unsafe-url\' />\n<meta name="pocket-site-verification" content="ed24b2b9721edf0a282c5b4a3232c4" />\n<meta name="biu" content="https://dr.habracdn.net/habr/6001a5d9/images/">\n\n<style type="text/css">\n  @font-face{font-family:\'Fira Sans\';font-style:normal;font-weight:500;src:url(https://dr.habracdn.net/habr/6001a5d9/fonts/FiraSans/firaSans-medium.eot);src:local("Fira Sans Medium"),local("FiraSans-Medium"),url(https://dr.habracdn.net/habr/6001a5d9/fonts/FiraSans/firaSans-medium.eot?#iefix) format("embedded-opentype"),url(https://dr.habracdn.net/habr/6001a5d9/fonts/FiraSans/firaSans-medium.woff2) format("woff2"),url(https://dr.habrac

In [4]:
soup = BeautifulSoup(res.text, 'html.parser')

In [5]:
news_blocks = soup.find_all('div', class_='page__body')

In [6]:
dates = list(map(lambda x: x.find_all('span', class_='post__time'), news_blocks))

In [7]:
headings = list(map(lambda x: x.find_all('h2', class_='post__title'), news_blocks))

In [8]:
all_titles = []
for heading in headings:
    for _ in heading:
        headings = _.find('a').get('href')
        all_titles.append(headings)
print(all_titles)

['https://habr.com/ru/post/446488/', 'https://habr.com/ru/post/340302/', 'https://habr.com/ru/post/450834/', 'https://habr.com/ru/post/521646/', 'https://habr.com/ru/company/fetchee/blog/316558/', 'https://habr.com/ru/company/pt/blog/210772/', 'https://habr.com/ru/post/414221/', 'https://habr.com/ru/company/click/blog/465277/', 'https://habr.com/ru/post/121815/', 'https://habr.com/ru/post/253439/', 'https://habr.com/ru/post/252379/', 'https://habr.com/ru/post/444876/', 'https://habr.com/ru/post/349942/', 'https://habr.com/ru/post/504900/', 'https://habr.com/ru/post/505888/', 'https://habr.com/ru/post/262991/', 'https://habr.com/ru/post/169409/', 'https://habr.com/ru/post/60430/', 'https://habr.com/ru/post/127584/', 'https://habr.com/ru/post/273253/']


In [9]:
news = pd.DataFrame()

for title in all_titles:
    soup = BeautifulSoup(requests.get(title).text, 'html.parser')
    time.sleep(0.3)
    date = soup.find('span', class_='post__time').get('data-time_published')
    title = soup.find('span', class_='post__title-text').text
    row = {'дата': date, 'заголовок': title, 'ссылка': title}
    news = pd.concat([news, pd.DataFrame([row])]) 
news

,дата,заголовок,ссылка
0,2019-04-03T12:21Z,"Правда про парсинг сайтов, или «все интернет-м...","Правда про парсинг сайтов, или «все интернет-м..."
0,2017-10-17T11:04Z,Парсинг сайтов: как с точки зрения закона выгл...,Парсинг сайтов: как с точки зрения закона выгл...
0,2019-05-07T11:51Z,Парсинг сайтов — а это вообще легально в России?,Парсинг сайтов — а это вообще легально в России?
0,2020-10-01T14:17Z,Как выбрать решение для парсинга сайтов: класс...,Как выбрать решение для парсинга сайтов: класс...
0,2016-12-01T09:46Z,Пример использования Product API от Fetchee дл...,Пример использования Product API от Fetchee дл...
0,2016-03-03T13:39Z,Теория и практика парсинга исходников с помощь...,Теория и практика парсинга исходников с помощь...
0,2018-06-18T20:42Z,Парсинг и работа с Codable в Swift 4,Парсинг и работа с Codable в Swift 4
0,2019-08-28T11:12Z,Парсинг и анализ семантики для SEO: 5 бесплатн...,Парсинг и анализ семантики для SEO: 5 бесплатн...
0,2011-06-14T15:24Z,Парсинг на Pуthon. Как собрать архив Голубятен,Парсинг на Pуthon. Как собрать архив Голубятен
0,2015-03-18T14:52Z,Продвинутый парсинг веб-сайтов с Mechanize,Продвинутый парсинг веб-сайтов с Mechanize


### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

## Задание 2.
### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [10]:
import json

In [11]:
email = ['xxx@x.ru', 'yyy@y.com']
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
params = {'emailAddresses': email}
headers = {'Vaar-Version' : '0', 'Vaar-Header-App-Product' : 'hackcheck-web-avast'}
req = requests.post(url, data = json.dumps(params), headers = headers)

In [12]:
dict_ = req.json()

In [13]:
list_email = pd.DataFrame()

In [14]:
for i,j in dict_.items():
      if i == 'summary':
        for email_, _ in dict_['summary'].items(): 
            list_ = _['breaches']
            for item in list_:
                date = (dict_['breaches'][str(item)]['publishDate'])
                source = (dict_['breaches'][str(item)]['site']) 
                description = (dict_['breaches'][str(item)]['description'])
                row = {'почта': email_, 'дата утечки': date, 'источник утечки': source, 'описание утечки': description}
                list_email = pd.concat([list_email, pd.DataFrame([row])])

In [15]:
list_email

,почта,дата утечки,источник утечки,описание утечки
0,xxx@x.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
0,xxx@x.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
0,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
0,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
0,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
0,yyy@y.com,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
0,yyy@y.com,2019-02-21T00:00:00Z,www.dangdang.com,"This is a list of email addresses only, and as..."


### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>